In [47]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
password = os.getenv('DB_CONNECTION_PASS')
db_name = 'forge_project2'

In [49]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password=password, database=db_name)
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

video = pd.read_csv('../sqldata/sql_video.csv')
channel = pd.read_csv('../sqldata/sql_channel.csv')
subscription = pd.read_csv('../sqldata/sql_subscription.csv')
view = pd.read_csv('../sqldata/sql_view.csv')
user = pd.read_csv('../sqldata/sql_user.csv')
owner = pd.read_csv('../sqldata/sql_owner.csv')
video_table = 'videos'
channel_table = 'channels'
subscription_table = 'subscriptions'
view_table = 'views'
user_table = 'users'
owner_table = 'owners'
video.to_sql(name=video_table, con=connection, if_exists='replace', index=False)
channel.to_sql(name=channel_table, con=connection, if_exists='replace', index=False)
subscription.to_sql(name=subscription_table, con=connection, if_exists='replace', index=False)
view.to_sql(name=view_table, con=connection, if_exists='replace', index=False)
user.to_sql(name=user_table, con=connection, if_exists='replace', index=False)
owner.to_sql(name=owner_table, con=connection, if_exists='replace', index=False)

10

### Age group engagement

In [58]:
query1 = """
SELECT
  CONCAT(
    age_group * 10 + 1,
    '-',
    age_group * 10 + 10
  ) AS 'Age Range',
  AVG(v.`Duration Watched`) AS 'Average Watch Time'
FROM (
  SELECT 
    u.`User ID`,
    FLOOR((2025 - u.`Birth Year`) / 10) AS age_group
  FROM Users u
) grouped_users
JOIN Views v ON grouped_users.`User ID` = v.`User ID`
GROUP BY age_group
ORDER BY age_group;
"""
result_df = pd.read_sql(query1, con=connection)
result_df

,Age Range,Average Watch Time
0,11-20,2974.3333
1,21-30,3520.6522
2,31-40,3634.4286
3,41-50,4658.3571
4,51-60,3832.0000


### Monthly Upload Count

In [63]:
query2 = """

SELECT * FROM VIDEOS LIMIT 1;

"""
result2_df = pd.read_sql(query2, con=connection)
result2_df

,Video ID,Title,Channel ID,Category,Duration,Rating,Upload Date
0,1,Life of Bees: Full Documentary,8,Documentary,5510,3.15,1/2/2025


In [75]:
query2 = """

SELECT MONTH(STR_TO_DATE(v.`Upload Date`, '%%c/%%e/%%Y')) AS Month,
COUNT(*) AS 'Number of Uploads'
FROM Videos v
GROUP BY Month
ORDER BY Month;


"""
result2_df = pd.read_sql(query2, con=connection)
result2_df

,Month,Number of Uploads
0,1,14
1,2,11
2,3,13
3,4,11
4,5,11


## When most subscriptions start (particular day of the week, some month)

### By Month

In [86]:
query3 = """

SELECT
  SUBSTRING(s.`Subscription Date`, 6, 2) AS Month,
  COUNT(*) AS 'Count of Subscriptions Started'
FROM Subscriptions s
GROUP BY Month
ORDER BY Month;



"""
result3_df = pd.read_sql(query3, con=connection)
result3_df

,Month,Count of Subscriptions Started
0,01,4
1,02,5
2,03,3
3,04,4
4,05,5
5,06,5
6,07,4
7,08,3
8,09,3
9,10,5


### By Day

In [95]:
query4 = """
SELECT DAYNAME(`Subscription Date`) AS `Day of the Week`, COUNT(*) AS `Number of Subscriptions Started`
FROM Subscriptions
GROUP BY `Day of the Week`
ORDER BY `Number of Subscriptions Started` DESC;
"""

result4_df = pd.read_sql(query4, con=connection)
result4_df

,Day of the Week,Number of Subscriptions Started
0,Monday,9
1,Thursday,9
2,Friday,7
3,Saturday,7
4,Wednesday,6
5,Tuesday,6
6,Sunday,6


## When most views take place (particular day of the week, some month)
Also can look at average duration watched as well


#### Month

In [97]:
query5 = """

SELECT
  SUBSTRING(v.`View Date`, 6, 2) AS Month,
  COUNT(*) AS 'Number of Views'
FROM Views v
GROUP BY Month
ORDER BY Month;


"""
result5_df = pd.read_sql(query5, con=connection)
result5_df

,Month,Number of Views
0,01,10
1,02,16
2,03,10
3,04,10
4,05,11
5,12,13


### Day

In [98]:
query6 = """
SELECT DAYNAME(v.`View Date`) AS `Day of the Week`, COUNT(*) AS `Number of Views`
FROM Views v
GROUP BY `Day of the Week`
ORDER BY `Number of Views` DESC;
"""

result6_df = pd.read_sql(query6, con=connection)
result6_df

,Day of the Week,Number of Views
0,Friday,15
1,Sunday,13
2,Saturday,12
3,Thursday,11
4,Tuesday,7
5,Wednesday,6
6,Monday,6


### Average Duration Watched Per Month

In [101]:
query7 = """
SELECT
  SUBSTRING(v.`View Date`, 6, 2) AS Month,
  AVG(v.`Duration Watched`) AS 'Average Watch Time'
FROM Views v
GROUP BY Month
ORDER BY Month;
"""

result7_df = pd.read_sql(query7, con=connection)
result7_df

,Month,Average Watch Time
0,01,4381.1000
1,02,3746.9375
2,03,4202.4000
3,04,3870.8000
4,05,2853.4545
5,12,3743.1538


### Average Duration Watched Per Day

In [ ]:
query8 = """
SELECT DAYNAME(v.`View Date`) AS `Day of the Week`, AVG(v.`Duration Watched`) AS `Average Watch Time`
FROM Views v
GROUP BY `Day of the Week`
ORDER BY `Average Watch Time` DESC;
"""

result8_df = pd.read_sql(query8, con=connection)
result8_df

,Day of the Week,Average Watch Time
0,Monday,5217.6667
1,Wednesday,4408.0000
2,Friday,4134.0667
3,Saturday,3862.0000
4,Tuesday,3540.4286
5,Thursday,3163.9091
6,Sunday,2988.3077


In [26]:
query9 = """
SELECT u.`Country`, AVG(v.`Rating`) AS `Average Rating`
FROM Users u
JOIN Subscriptions s ON u.`User ID` = s.`User ID`
JOIN Channels c ON s.`Channel ID` = c.`Channel ID`
JOIN Videos v ON c.`Channel ID` = v.`Channel ID`
GROUP BY u.`Country`
ORDER BY `Average Rating` DESC;
"""

result9_df = pd.read_sql(query9, con=connection)
result9_df

,Country,Average Rating
0,South Korea,4.244000
1,Italy,4.208571
2,Mexico,3.876522
3,Singapore,3.835581
4,India,3.814091
5,Netherlands,3.801852
6,United States,3.753929
7,China,3.753913
8,Germany,3.729500
9,Switzerland,3.722500
